In [6]:
import pandas as pd #
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from scipy.stats import zscore

csv_path = "./data_raw/final_moving_avg_devID109_test.csv"
#device = "103"
#save_path = f"./data/dev{device}_prepared.csv"
save_path = f"./data/final_moving_avg_devID109_test_prepared.csv"

df = pd.read_csv(csv_path)

In [5]:
df.head()

,timestamp,devID,temp,humi,soil,rain,geo,rain_ma_1h,rain_ma_6h,rain_ma_12h,rain_ma_24h,soil_ma_4h,soil_ma_8h,soil_ma_12h,soil_ma_16h,geo_ma_2min,geo_ma_4min,geo_ma_6min,geo_ma_8min,label
0,11/16/2025 6:10,Node 109,21.70,93.45,35.6,0.0,0.34,0.0,0.0,0.0,0.0,35.809355,35.956271,36.160034,36.212209,3.533333,5.100000,4.542500,3.414545,NaN
1,11/16/2025 6:11,Node 109,21.70,93.30,35.6,0.0,14.56,0.0,0.0,0.0,0.0,35.808091,35.954523,36.158568,36.211563,5.106667,8.886000,5.898750,4.693636,NaN
2,11/16/2025 6:12,Node 109,21.75,93.25,35.6,0.0,7.42,0.0,0.0,0.0,0.0,35.807120,35.953350,36.157103,36.210918,10.990000,6.516000,6.572500,5.832000,NaN
3,11/16/2025 6:13,Node 109,21.80,93.30,35.6,0.0,0.35,0.0,0.0,0.0,0.0,35.806149,35.952178,36.154797,36.210274,3.885000,4.618000,7.457143,5.496000,NaN
4,11/16/2025 6:13,Node 109,21.80,93.20,35.6,0.0,0.27,0.0,0.0,0.0,0.0,35.805484,35.951589,36.154171,36.209632,2.680000,3.893333,6.558750,5.020909,NaN


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2540 entries, 0 to 2539
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   timestamp    2540 non-null   object 
 1   devID        2540 non-null   object 
 2   temp         2540 non-null   float64
 3   humi         2540 non-null   float64
 4   soil         2540 non-null   float64
 5   rain         2540 non-null   float64
 6   geo          2540 non-null   float64
 7   rain_ma_1h   2540 non-null   float64
 8   rain_ma_6h   2540 non-null   float64
 9   rain_ma_12h  2540 non-null   float64
 10  rain_ma_24h  2540 non-null   float64
 11  soil_ma_4h   2540 non-null   float64
 12  soil_ma_8h   2540 non-null   float64
 13  soil_ma_12h  2540 non-null   float64
 14  soil_ma_16h  2540 non-null   float64
 15  geo_ma_2min  2540 non-null   float64
 16  geo_ma_4min  2540 non-null   float64
 17  geo_ma_6min  2540 non-null   float64
 18  geo_ma_8min  2540 non-null   float64
 19  label 

In [8]:
df.describe()

,temp,humi,soil,rain,geo,rain_ma_1h,rain_ma_6h,rain_ma_12h,rain_ma_24h,soil_ma_4h,soil_ma_8h,soil_ma_12h,soil_ma_16h,geo_ma_2min,geo_ma_4min,geo_ma_6min,geo_ma_8min
count,2540.000000,2540.000000,2540.000000,2540.000000,2540.000000,2540.000000,2540.000000,2540.000000,2540.000000,2540.000000,2540.000000,2540.000000,2540.000000,2540.000000,2540.000000,2540.000000,2540.000000
mean,26.700984,84.822441,35.998819,0.539272,3.301402,0.531563,0.416437,0.198795,0.099429,35.714055,35.346769,35.263698,35.304817,3.262953,3.296746,3.296370,3.292017
std,3.327816,13.488397,3.170013,2.958587,6.415728,2.002547,1.060165,0.501684,0.251435,2.587445,1.412251,0.912019,0.738720,5.420886,4.615207,4.221661,3.951139
min,21.700000,52.100000,34.000000,0.000000,-10.940000,0.000000,0.000000,0.000000,0.000000,34.084712,34.131952,34.184304,34.227596,-1.650000,-0.126000,-0.006667,0.285000
25%,24.000000,71.637500,34.300000,0.000000,0.360000,0.000000,0.000000,0.000000,0.000000,34.300000,34.347596,34.472581,34.640683,0.405000,0.520000,0.684152,0.931364
50%,24.850000,91.900000,34.750000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,34.883307,35.072828,35.190104,35.290953,0.705833,1.415833,1.645238,1.794610
75%,29.500000,95.112500,35.500000,0.000000,2.190000,0.000000,0.010742,0.005217,0.002445,35.538143,35.644091,35.738828,35.861603,3.155833,3.744375,4.132875,4.332341
max,34.300000,98.100000,46.600000,31.990000,32.000000,13.636786,3.557418,1.700212,0.850776,44.101025,41.049708,38.530970,37.434320,31.420000,27.736667,26.217500,24.972000


In [9]:
df['devID'] = df['devID'].str.replace('Node ', '', regex=False)

In [11]:
#df.drop(columns=['device'], inplace=True)

In [14]:
for col in ['soil', 'rain', 'temp', 'humi', 'geo', 'devID']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df


,timestamp,devID,temp,humi,soil,rain,geo,rain_ma_1h,rain_ma_6h,rain_ma_12h,rain_ma_24h,soil_ma_4h,soil_ma_8h,soil_ma_12h,soil_ma_16h,geo_ma_2min,geo_ma_4min,geo_ma_6min,geo_ma_8min,label
0,2025-11-16 06:10:00,109,21.70,93.45,35.6,0.00,0.34,0.000000,0.000000,0.000000,0.000000,35.809355,35.956271,36.160034,36.212209,3.533333,5.100000,4.542500,3.414545,NaN
1,2025-11-16 06:11:00,109,21.70,93.30,35.6,0.00,14.56,0.000000,0.000000,0.000000,0.000000,35.808091,35.954523,36.158568,36.211563,5.106667,8.886000,5.898750,4.693636,NaN
2,2025-11-16 06:12:00,109,21.75,93.25,35.6,0.00,7.42,0.000000,0.000000,0.000000,0.000000,35.807120,35.953350,36.157103,36.210918,10.990000,6.516000,6.572500,5.832000,NaN
3,2025-11-16 06:13:00,109,21.80,93.30,35.6,0.00,0.35,0.000000,0.000000,0.000000,0.000000,35.806149,35.952178,36.154797,36.210274,3.885000,4.618000,7.457143,5.496000,NaN
4,2025-11-16 06:13:00,109,21.80,93.20,35.6,0.00,0.27,0.000000,0.000000,0.000000,0.000000,35.805484,35.951589,36.154171,36.209632,2.680000,3.893333,6.558750,5.020909,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2535,2025-11-17 18:21:00,109,23.80,97.95,41.9,0.28,0.54,0.240563,3.170836,1.699166,0.850609,43.076996,40.988889,38.497883,37.406198,0.415000,0.382000,0.366667,0.384444,NaN
2536,2025-11-17 18:21:00,109,23.80,98.00,41.9,0.28,3.74,0.241111,3.163307,1.697400,0.850255,43.072538,40.990661,38.502114,37.410351,1.523333,0.941667,0.848571,0.720000,NaN
2537,2025-11-17 18:22:00,109,23.80,98.00,41.9,0.28,3.74,0.241644,3.131462,1.699863,0.849901,43.055871,41.005837,38.517061,37.417375,2.673333,1.740000,1.341429,1.052000,NaN
2538,2025-11-17 18:23:00,109,23.80,98.00,41.9,0.28,0.47,0.230137,3.111775,1.700212,0.850074,43.026806,41.034503,38.526775,37.427290,2.105000,1.756000,1.232500,1.127778,NaN


In [15]:
df.columns = df.columns.str.strip()
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df = df.dropna(subset=['timestamp', 'geo'])

df = df[df['soil'] > 0]
df = df[df['rain'] >= 0]
df = df[df['humi'] > 0]
df = df[(df['geo'] >= 0) & (df['geo'] < 40)]
df = df[df['temp'] > 0]
df = df[(df['devID'] > 0) & (df['devID'] < 1000)]
df

,timestamp,devID,temp,humi,soil,rain,geo,rain_ma_1h,rain_ma_6h,rain_ma_12h,rain_ma_24h,soil_ma_4h,soil_ma_8h,soil_ma_12h,soil_ma_16h,geo_ma_2min,geo_ma_4min,geo_ma_6min,geo_ma_8min,label
0,2025-11-16 06:10:00,109,21.70,93.45,35.6,0.00,0.34,0.000000,0.000000,0.000000,0.000000,35.809355,35.956271,36.160034,36.212209,3.533333,5.100000,4.542500,3.414545,NaN
1,2025-11-16 06:11:00,109,21.70,93.30,35.6,0.00,14.56,0.000000,0.000000,0.000000,0.000000,35.808091,35.954523,36.158568,36.211563,5.106667,8.886000,5.898750,4.693636,NaN
2,2025-11-16 06:12:00,109,21.75,93.25,35.6,0.00,7.42,0.000000,0.000000,0.000000,0.000000,35.807120,35.953350,36.157103,36.210918,10.990000,6.516000,6.572500,5.832000,NaN
3,2025-11-16 06:13:00,109,21.80,93.30,35.6,0.00,0.35,0.000000,0.000000,0.000000,0.000000,35.806149,35.952178,36.154797,36.210274,3.885000,4.618000,7.457143,5.496000,NaN
4,2025-11-16 06:13:00,109,21.80,93.20,35.6,0.00,0.27,0.000000,0.000000,0.000000,0.000000,35.805484,35.951589,36.154171,36.209632,2.680000,3.893333,6.558750,5.020909,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2535,2025-11-17 18:21:00,109,23.80,97.95,41.9,0.28,0.54,0.240563,3.170836,1.699166,0.850609,43.076996,40.988889,38.497883,37.406198,0.415000,0.382000,0.366667,0.384444,NaN
2536,2025-11-17 18:21:00,109,23.80,98.00,41.9,0.28,3.74,0.241111,3.163307,1.697400,0.850255,43.072538,40.990661,38.502114,37.410351,1.523333,0.941667,0.848571,0.720000,NaN
2537,2025-11-17 18:22:00,109,23.80,98.00,41.9,0.28,3.74,0.241644,3.131462,1.699863,0.849901,43.055871,41.005837,38.517061,37.417375,2.673333,1.740000,1.341429,1.052000,NaN
2538,2025-11-17 18:23:00,109,23.80,98.00,41.9,0.28,0.47,0.230137,3.111775,1.700212,0.850074,43.026806,41.034503,38.526775,37.427290,2.105000,1.756000,1.232500,1.127778,NaN


In [16]:
df.columns = df.columns.str.strip()
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df = df.dropna(subset=['timestamp', 'geo'])  # ต้องมี timestamp และ geo

for col in ['soil', 'rain', 'temp', 'humi', 'geo']:

    df[col] = pd.to_numeric(df[col], errors='coerce')
df

,timestamp,devID,temp,humi,soil,rain,geo,rain_ma_1h,rain_ma_6h,rain_ma_12h,rain_ma_24h,soil_ma_4h,soil_ma_8h,soil_ma_12h,soil_ma_16h,geo_ma_2min,geo_ma_4min,geo_ma_6min,geo_ma_8min,label
0,2025-11-16 06:10:00,109,21.70,93.45,35.6,0.00,0.34,0.000000,0.000000,0.000000,0.000000,35.809355,35.956271,36.160034,36.212209,3.533333,5.100000,4.542500,3.414545,NaN
1,2025-11-16 06:11:00,109,21.70,93.30,35.6,0.00,14.56,0.000000,0.000000,0.000000,0.000000,35.808091,35.954523,36.158568,36.211563,5.106667,8.886000,5.898750,4.693636,NaN
2,2025-11-16 06:12:00,109,21.75,93.25,35.6,0.00,7.42,0.000000,0.000000,0.000000,0.000000,35.807120,35.953350,36.157103,36.210918,10.990000,6.516000,6.572500,5.832000,NaN
3,2025-11-16 06:13:00,109,21.80,93.30,35.6,0.00,0.35,0.000000,0.000000,0.000000,0.000000,35.806149,35.952178,36.154797,36.210274,3.885000,4.618000,7.457143,5.496000,NaN
4,2025-11-16 06:13:00,109,21.80,93.20,35.6,0.00,0.27,0.000000,0.000000,0.000000,0.000000,35.805484,35.951589,36.154171,36.209632,2.680000,3.893333,6.558750,5.020909,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2535,2025-11-17 18:21:00,109,23.80,97.95,41.9,0.28,0.54,0.240563,3.170836,1.699166,0.850609,43.076996,40.988889,38.497883,37.406198,0.415000,0.382000,0.366667,0.384444,NaN
2536,2025-11-17 18:21:00,109,23.80,98.00,41.9,0.28,3.74,0.241111,3.163307,1.697400,0.850255,43.072538,40.990661,38.502114,37.410351,1.523333,0.941667,0.848571,0.720000,NaN
2537,2025-11-17 18:22:00,109,23.80,98.00,41.9,0.28,3.74,0.241644,3.131462,1.699863,0.849901,43.055871,41.005837,38.517061,37.417375,2.673333,1.740000,1.341429,1.052000,NaN
2538,2025-11-17 18:23:00,109,23.80,98.00,41.9,0.28,0.47,0.230137,3.111775,1.700212,0.850074,43.026806,41.034503,38.526775,37.427290,2.105000,1.756000,1.232500,1.127778,NaN


In [17]:
df = df.sort_values('timestamp').reset_index(drop=True)
df = df.set_index('timestamp')
df = df.reset_index() 

In [18]:
df.head()

,timestamp,devID,temp,humi,soil,rain,geo,rain_ma_1h,rain_ma_6h,rain_ma_12h,rain_ma_24h,soil_ma_4h,soil_ma_8h,soil_ma_12h,soil_ma_16h,geo_ma_2min,geo_ma_4min,geo_ma_6min,geo_ma_8min,label
0,2025-11-16 06:10:00,109,21.70,93.45,35.6,0.0,0.34,0.0,0.0,0.0,0.0,35.809355,35.956271,36.160034,36.212209,3.533333,5.100000,4.542500,3.414545,NaN
1,2025-11-16 06:11:00,109,21.70,93.30,35.6,0.0,14.56,0.0,0.0,0.0,0.0,35.808091,35.954523,36.158568,36.211563,5.106667,8.886000,5.898750,4.693636,NaN
2,2025-11-16 06:12:00,109,21.75,93.25,35.6,0.0,7.42,0.0,0.0,0.0,0.0,35.807120,35.953350,36.157103,36.210918,10.990000,6.516000,6.572500,5.832000,NaN
3,2025-11-16 06:13:00,109,21.80,93.30,35.6,0.0,0.35,0.0,0.0,0.0,0.0,35.806149,35.952178,36.154797,36.210274,3.885000,4.618000,7.457143,5.496000,NaN
4,2025-11-16 06:13:00,109,21.80,93.20,35.6,0.0,0.27,0.0,0.0,0.0,0.0,35.805484,35.951589,36.154171,36.209632,2.680000,3.893333,6.558750,5.020909,NaN


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2535 entries, 0 to 2534
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   timestamp    2535 non-null   datetime64[ns]
 1   devID        2535 non-null   int64         
 2   temp         2535 non-null   float64       
 3   humi         2535 non-null   float64       
 4   soil         2535 non-null   float64       
 5   rain         2535 non-null   float64       
 6   geo          2535 non-null   float64       
 7   rain_ma_1h   2535 non-null   float64       
 8   rain_ma_6h   2535 non-null   float64       
 9   rain_ma_12h  2535 non-null   float64       
 10  rain_ma_24h  2535 non-null   float64       
 11  soil_ma_4h   2535 non-null   float64       
 12  soil_ma_8h   2535 non-null   float64       
 13  soil_ma_12h  2535 non-null   float64       
 14  soil_ma_16h  2535 non-null   float64       
 15  geo_ma_2min  2535 non-null   float64       
 16  geo_ma

In [20]:
df.describe()

,timestamp,devID,temp,humi,soil,rain,geo,rain_ma_1h,rain_ma_6h,rain_ma_12h,rain_ma_24h,soil_ma_4h,soil_ma_8h,soil_ma_12h,soil_ma_16h,geo_ma_2min,geo_ma_4min,geo_ma_6min,geo_ma_8min
count,2535,2535.0,2535.000000,2535.000000,2535.000000,2535.000000,2535.000000,2535.000000,2535.000000,2535.000000,2535.000000,2535.000000,2535.000000,2535.000000,2535.000000,2535.000000,2535.000000,2535.000000,2535.000000
mean,2025-11-16 23:47:25.088757504,109.0,26.696765,84.839961,36.000907,0.540335,3.318067,0.532612,0.417254,0.199185,0.099624,35.715328,35.347049,35.263572,35.304480,3.268501,3.292424,3.294665,3.291949
min,2025-11-16 06:10:00,109.0,21.700000,52.100000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,34.084712,34.131952,34.184304,34.227596,-1.290000,-0.126000,-0.006667,0.285000
25%,2025-11-16 14:37:30,109.0,24.000000,71.675000,34.300000,0.000000,0.360000,0.000000,0.000000,0.000000,0.000000,34.300000,34.347500,34.472194,34.640607,0.405000,0.520000,0.684018,0.930909
50%,2025-11-16 23:30:00,109.0,24.800000,91.900000,34.750000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,34.881429,35.071066,35.188064,35.288704,0.710000,1.414000,1.642500,1.787143
75%,2025-11-17 08:37:30,109.0,29.500000,95.125000,35.500000,0.000000,2.190000,0.000000,0.010742,0.005217,0.002445,35.538279,35.644298,35.739066,35.862011,3.158333,3.726667,4.108571,4.332727
max,2025-11-17 18:24:00,109.0,34.300000,98.100000,46.600000,31.990000,32.000000,13.636786,3.557418,1.700212,0.850776,44.101025,41.049708,38.530970,37.434320,31.420000,27.736667,26.217500,24.972000
std,NaN,0.0,3.325174,13.479358,3.172703,2.961407,6.409452,2.004383,1.061051,0.502102,0.251644,2.589751,1.413448,0.912599,0.739012,5.423248,4.616908,4.224603,3.954424


In [21]:
df['devID'].value_counts()

devID
109    2535
Name: count, dtype: int64

In [22]:

df.to_csv(f"{save_path}", index=False)

print("✅ บันทึกเรียบร้อยแล้ว")

✅ บันทึกเรียบร้อยแล้ว
